In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import optuna

seeds = 1234
def set_seeds(seed=seeds):
    """Set seeds for reproducibility."""
    np.random.seed(seed)
    random.seed(seed)
# Set seeds
set_seeds()

# Import train set
df = pd.read_pickle(r"train_enc.pkl")

# Separate labels and target
X, y =  df.drop(columns="left", axis = 1), df["left"]


In [2]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RepeatedStratifiedKFold

from imblearn.pipeline import make_pipeline, Pipeline
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.combine import SMOTEENN
from lightgbm import LGBMClassifier

from sklearn.metrics import fbeta_score, make_scorer, precision_score, recall_score

def f2_measure(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=2)

f2_score = make_scorer(f2_measure)    

# Get variable names
bi_vars = [col for col in X.columns if X[col].nunique() == 2]
num_vars = [col for col in X.columns if (X[col].dtype in ["int8", "float32"]) & (col not in bi_vars)]
cat_vars = [col for col in X.columns if (col not in num_vars) & (col not in bi_vars)]

# Preprocessing recipe
preprocess = ColumnTransformer(
    transformers=[
        ("scaler", MinMaxScaler(), num_vars),
        ("ohe", OneHotEncoder(), cat_vars)
    ], remainder = "passthrough"
)

# Resampling
smoteen = SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='majority'), random_state=seeds)

clf = LGBMClassifier(random_state=seeds)
pipe_final = make_pipeline(preprocess, smoteen, clf)

## Tune 1

In [103]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,f1_score
import optuna


def objective(trial,data=X,target=y):
    
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=seeds)
    params = {
                "verbosity": -1,
                "boosting_type": "gbdt",
                "num_leaves": trial.suggest_int("num_leaves", 2, 256),
                "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 0.9),
                "colsample_bynode": trial.suggest_float("colsample_bynode", 0.2, 0.9),
    }
    clf = LGBMClassifier(random_state=seeds,scale_pos_weight=2, **params)
    model = make_pipeline(preprocess, clf)
    model.fit(train_x, train_y)
    predictions = model.predict_proba(test_x)[:,1]
    f1 = f1_score(test_y, predictions, average='macro')
    return f1

In [104]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-02-17 16:51:21,740] A new study created in memory with name: no-name-a8bfa4e4-ed9c-4617-a9aa-1e33cb23c833
[W 2022-02-17 16:51:21,748] Trial 0 failed because of the following error: ValueError('Pipeline.fit does not accept the eval_set parameter. You can pass parameters to specific steps of your pipeline using the stepname__parameter format, e.g. `Pipeline.fit(X, y, logisticregression__sample_weight=sample_weight)`.')
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\env_full\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Eric\AppData\Local\Temp/ipykernel_4756/3310503327.py", line 22, in objective
    model.fit(train_x, train_y, eval_set=[(test_x, test_y)],
  File "C:\ProgramData\Anaconda3\envs\env_full\lib\site-packages\imblearn\pipeline.py", line 267, in fit
    fit_params_steps = self._check_fit_params(**fit_params)
  File "C:\ProgramData\Anaconda3\envs\env_full\lib\site-packages\

ValueError: Pipeline.fit does not accept the eval_set parameter. You can pass parameters to specific steps of your pipeline using the stepname__parameter format, e.g. `Pipeline.fit(X, y, logisticregression__sample_weight=sample_weight)`.

## Tune

In [88]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import optuna


def objective(trial,data=X,target=y):
    
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=seeds)
    params = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    clf = LGBMClassifier(random_state=seeds,scale_pos_weight=2, **params)
    model = make_pipeline(preprocess, clf)
    model.fit(train_x, train_y)
    predictions = model.predict_proba(test_x)[:,1]
    auc = roc_auc_score(test_y, predictions)
    return auc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-02-17 16:33:16,291] A new study created in memory with name: no-name-8b3d7cf2-c870-4fd0-82fa-9a41e08807ab


[LightGBM] [Warning] feature_fraction is set=0.8267236727110189, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8267236727110189
[LightGBM] [Warning] lambda_l1 is set=0.06514575898316952, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.06514575898316952
[LightGBM] [Warning] bagging_fraction is set=0.8349946090451172, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8349946090451172
[LightGBM] [Warning] lambda_l2 is set=1.9751838457968587e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.9751838457968587e-07
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


[I 2022-02-17 16:33:16,722] Trial 0 finished with value: 0.924957720277355 and parameters: {'lambda_l1': 0.06514575898316952, 'lambda_l2': 1.9751838457968587e-07, 'num_leaves': 131, 'feature_fraction': 0.8267236727110189, 'bagging_fraction': 0.8349946090451172, 'bagging_freq': 3, 'min_child_samples': 98}. Best is trial 0 with value: 0.924957720277355.


[LightGBM] [Warning] feature_fraction is set=0.43021982955569393, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.43021982955569393
[LightGBM] [Warning] lambda_l1 is set=3.810155143368929, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.810155143368929
[LightGBM] [Warning] bagging_fraction is set=0.6903749522871129, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6903749522871129
[LightGBM] [Warning] lambda_l2 is set=0.0001065975764698808, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0001065975764698808
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2022-02-17 16:33:17,063] Trial 1 finished with value: 0.9248414510400812 and parameters: {'lambda_l1': 3.810155143368929, 'lambda_l2': 0.0001065975764698808, 'num_leaves': 174, 'feature_fraction': 0.43021982955569393, 'bagging_fraction': 0.6903749522871129, 'bagging_freq': 1, 'min_child_samples': 53}. Best is trial 0 with value: 0.924957720277355.


...

[LightGBM] [Warning] feature_fraction is set=0.7068932769586695, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7068932769586695
[LightGBM] [Warning] lambda_l1 is set=0.031535301069265324, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.031535301069265324
[LightGBM] [Warning] bagging_fraction is set=0.774802932280211, subsample=1.0 will be ignored. Current value: bagging_fraction=0.774802932280211
[LightGBM] [Warning] lambda_l2 is set=0.30243372528938245, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.30243372528938245
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6


[I 2022-02-17 16:33:41,094] Trial 48 finished with value: 0.9201357179096905 and parameters: {'lambda_l1': 0.031535301069265324, 'lambda_l2': 0.30243372528938245, 'num_leaves': 179, 'feature_fraction': 0.7068932769586695, 'bagging_fraction': 0.774802932280211, 'bagging_freq': 6, 'min_child_samples': 5}. Best is trial 42 with value: 0.928006088280061.


[LightGBM] [Warning] feature_fraction is set=0.8408951636735417, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8408951636735417
[LightGBM] [Warning] lambda_l1 is set=0.12366898578824966, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.12366898578824966
[LightGBM] [Warning] bagging_fraction is set=0.7345990203069114, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7345990203069114
[LightGBM] [Warning] lambda_l2 is set=0.48330904379176376, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.48330904379176376
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


[I 2022-02-17 16:33:41,575] Trial 49 finished with value: 0.9246871300524269 and parameters: {'lambda_l1': 0.12366898578824966, 'lambda_l2': 0.48330904379176376, 'num_leaves': 95, 'feature_fraction': 0.8408951636735417, 'bagging_fraction': 0.7345990203069114, 'bagging_freq': 5, 'min_child_samples': 69}. Best is trial 42 with value: 0.928006088280061.


Number of finished trials: 50
Best trial: {'lambda_l1': 9.693520077339764, 'lambda_l2': 0.0011718408699389177, 'num_leaves': 115, 'feature_fraction': 0.7966576759605296, 'bagging_fraction': 0.6991081699843358, 'bagging_freq': 4, 'min_child_samples': 61}


In [90]:
study.best_trial.params

{'lambda_l1': 9.693520077339764,
 'lambda_l2': 0.0011718408699389177,
 'num_leaves': 115,
 'feature_fraction': 0.7966576759605296,
 'bagging_fraction': 0.6991081699843358,
 'bagging_freq': 4,
 'min_child_samples': 61}

In [108]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import optuna


def objective(trial,data=X,target=y):
    
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25,random_state=seeds)
    params = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    clf = LGBMClassifier(random_state=seeds,scale_pos_weight=2, **params)
    model = make_pipeline(preprocess,smoteen, clf)
    metric = make_scorer(f2_measure)
    # Evaluate model
    scores = cross_val_score(model,test_x, test_y, scoring=metric, cv=3, n_jobs=-1).mean()
    return scores

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-02-17 17:03:34,505] A new study created in memory with name: no-name-85c79d04-9b94-40ef-a297-62d851684703
[I 2022-02-17 17:03:34,732] Trial 0 finished with value: 0.7843229505356429 and parameters: {'lambda_l1': 1.0718804276943297e-08, 'lambda_l2': 6.313719211217128e-05, 'num_leaves': 13, 'feature_fraction': 0.835105825861003, 'bagging_fraction': 0.6690570960165985, 'bagging_freq': 3, 'min_child_samples': 48}. Best is trial 0 with value: 0.7843229505356429.
[I 2022-02-17 17:03:34,961] Trial 1 finished with value: 0.7830201351672801 and parameters: {'lambda_l1': 0.013826865927299446, 'lambda_l2': 0.00021482568347568632, 'num_leaves': 232, 'feature_fraction': 0.41284107871574893, 'bagging_fraction': 0.4152344107212189, 'bagging_freq': 4, 'min_child_samples': 42}. Best is trial 0 with value: 0.7843229505356429.
[I 2022-02-17 17:03:35,436] Trial 2 finished with value: 0.7812428822608769 and parameters: {'lambda_l1': 3.3036489348816736e-08, 'lambda_l2': 1.1902502356071903e-05, 'num_

Number of finished trials: 50
Best trial: {'lambda_l1': 9.372189667936706e-07, 'lambda_l2': 5.006422292802079, 'num_leaves': 207, 'feature_fraction': 0.5726199420672202, 'bagging_fraction': 0.9838013199012343, 'bagging_freq': 3, 'min_child_samples': 17}


## Results

In [113]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


test = pd.read_pickle(r"test_enc.pkl")
X_test, y_test =  test.drop(columns="left", axis = 1), test["left"]

params= {'lambda_l1': 9.693520077339764,
 'lambda_l2': 0.0011718408699389177,
 'num_leaves': 115,
 'feature_fraction': 0.7966576759605296,
 'bagging_fraction': 0.6991081699843358,
 'bagging_freq': 4,
 'min_child_samples': 61}

clf2 = LGBMClassifier(random_state=seeds,**params)

pipeline_final = make_pipeline(preprocess, smoteen, clf2)

pipeline_final.fit(X, y)
    # Obtain the predictions from our random forest model 
predicted = pipeline_final.predict(X_test)
    # Predict probabilities
probs = pipeline_final.predict_proba(X_test)
    # Print the ROC curve, classification report and confusion matrix
print(fbeta_score(y_test, predicted, beta=2))
print(classification_report(y_test, predicted))
print(confusion_matrix(y_test, predicted))

[LightGBM] [Warning] feature_fraction is set=0.7966576759605296, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7966576759605296
[LightGBM] [Warning] lambda_l1 is set=9.693520077339764, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.693520077339764
[LightGBM] [Warning] bagging_fraction is set=0.6991081699843358, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6991081699843358
[LightGBM] [Warning] lambda_l2 is set=0.0011718408699389177, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0011718408699389177
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
0.8283558629361835
              precision    recall  f1-score   support

           0       0.97      0.68      0.80      1351
           1       0.55      0.95      0.70       557

    accuracy                           0.76      1908
   macro avg       0.76      0.82      0.75      1908
weighted avg       0.85      